# Projeto: Web Scraping e Processamento de Letras do Vagalume
Este projeto tem como objetivo principal a coleta e o processamento de dados de letras musicais e discografias do site Vagalume. A base de dados final é construída através de técnicas de web scraping e organizada em formatos estruturados para posterior análise.

# Bibliotecas e dependências

In [ ]:
%pip install -r requirements.txt
import chardet
import re
import unicodedata
import requests
import bs4
import time
import json
import csv
import os
import pandas as pd

# Utilitários
EncodingManager: Gerencia problemas de codificação de caracteres

    detect_encoding: Detecta a codificação de um conteúdo binário

    process_content: Processa o conteúdo binário tratando problemas de encoding

In [3]:
class EncodingManager:
    """Componente para gerenciar problemas de encoding"""
    
    def detect_encoding(self, content):
        """Detecta a codificação de um conteúdo binário"""
        result = chardet.detect(content)
        return result['encoding']
        
    def process_content(self, content):
        """Processa o conteúdo binário tratando problemas de encoding"""
        encoding = self.detect_encoding(content)
        print(f"[DEBUG] Codificação detectada: {encoding}")
        
        sample = content[:100]
        print(f"[DEBUG] Bytes de amostra: {sample}")
        encodings_to_try = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
        for enc in encodings_to_try:
            try:
                decoded = sample.decode(enc)
                print(f"[DEBUG] Amostra decodificada com {enc}: {decoded}")
            except UnicodeDecodeError as e:
                print(f"[DEBUG] Decodificação de erros com {enc}: {e}")
        
        # Se não detectar encoding ou for desconhecido, usa utf-8 com fallback para latin-1
        if not encoding or encoding.lower() == 'unknown':
            try:
                return content.decode('utf-8')
            except UnicodeDecodeError:
                return content.decode('latin-1')
        
        # Caso contrário, usa o encoding detectado
        try:
            return content.decode(encoding)
        except UnicodeDecodeError:
            # Fallback para utf-8 ou latin-1
            try:
                return content.decode('utf-8')
            except UnicodeDecodeError:
                return content.decode('latin-1')
            

# Processadores
TextProcessor: Normaliza e processa textos extraídos

    normalize: Normaliza o texto da letra de música

In [5]:
class TextProcessor:
    """Componente para processar e normalizar textos"""
    
    def normalize(self, text):
        """Normaliza o texto removendo caracteres especiais, ajustando espaços, etc."""
        if not text:
            return ""
            
        # Remove espaços extras
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Normaliza caracteres Unicode
        text = unicodedata.normalize('NFKC', text)
        
        return text
        
    def tokenize(self, text):
        """Tokeniza o texto em palavras"""
        if not text:
            return []
            
        # Normaliza primeiro
        text = self.normalize(text)
        
        # Divide em tokens (palavras)
        tokens = re.findall(r'\b\w+\b', text.lower())
        
        return tokens

# Extratores
Componentes responsáveis por extrair dados de diferentes fontes:

ArtistExtractor: Extrai nomes de artistas de um arquivo texto

    get_artists: Lê a lista de artistas do arquivo

DiscographyExtractor: Extrai informações da discografia de um artista

    extract_discography: Extrai a discografia completa
    _extract_album_info: Extrai informações de um álbum específico
    _extract_tracks: Extrai a lista de músicas de um álbum

LyricsExtractor: Extrai letras de músicas

    extract_lyrics: Extrai a letra de uma música específica

In [6]:
class ArtistExtractor:
    """Componente para extrair informações de artistas de arquivos ou outras fontes"""
    
    def __init__(self, file_path='artistas.txt'):
        self.file_path = file_path
        
    def get_artists(self):
        """Lê artistas de um arquivo de texto"""
        try:
            return [linha.strip() for linha in open(self.file_path, encoding='utf-8')]
        except Exception as e:
            print(f"Erro ao carregar artistas: {e}")
            return []

In [8]:
class DiscographyExtractor:
    """Componente para extrair discografia de artistas de sites"""
    
    def __init__(self, encoding_manager=None):
        self.encoding_manager = encoding_manager
        self.base_url = 'https://www.vagalume.com.br/'
        
    def get_discography_url(self, artist):
        """Constrói a URL para a página de discografia do artista"""
        return f"{self.base_url}{artist}/discografia/"
        
    def extract_discography(self, artist):
        """Extrai a discografia completa de um artista"""
        url = self.get_discography_url(artist)
        
        try:
            # Faz a requisição
            req = requests.get(url)
            
            # Usa o gerenciador de encoding se disponível
            if self.encoding_manager:
                content = self.encoding_manager.process_content(req.content)
            else:
                content = req.content
                
            # Analisa o HTML
            soup = bs4.BeautifulSoup(content, "html.parser")

            # Extrai nome do artista
            artist_name = soup.find("h1", "darkBG long")
            if artist_name == None:
                artist_name = soup.find("h1", "darkBG")
            artist_name = artist_name.get_text().strip()

            # Extrai tags associadas ao artista
            artist_tags_list = soup.find("ul", "subHeaderTags h14")
            artist_tags = []

            if artist_tags_list:
                tag_items = artist_tags_list.find_all("a")
                for tag in tag_items:
                    tag_text = tag.get_text().strip()
                    artist_tags.append(tag_text)
            
            # Lista para guardar os álbuns
            discografia = []
            
            # Recupera os álbuns
            albuns = soup.find_all("div", "topLetrasWrapper")
            
            for album_div in albuns:
                album = self._extract_album_info(album_div)
                discografia.append(album)
                
            # Adiciona delay para evitar sobrecarga do servidor
            time.sleep(1.5)
            
            return discografia, artist_name, artist_tags
            
        except Exception as e:
            print(f"Erro ao extrair discografia de {artist}: {e}")
            return []
    
    def _extract_album_info(self, album_div):
        """Extrai informações de um álbum específico"""
        album = {}

        # Extrai título do álbum
        album_title_elem = album_div.find("h1", "albumTitle")
        if album_title_elem:
            album["album_title"] = album_title_elem.get_text().strip()
        else:
            album["album_title"] = None
        
        # Extrai ano e gravadora
        album_year_elem = album_div.find("p", "albumYear")
        album_record_elem = album_div.find("p", "albumRecord")
        
        if album_year_elem:
            year_text = album_year_elem.get_text().strip()
            # Separar o ano do resto usando regex
            match = re.match(r"(\d{4})(.*)", year_text)
            if match:
                album["year"] = match.group(1)
                # Se sobrou algo depois do ano, é a gravadora
                possible_label = match.group(2).strip()
                if possible_label and not album_record_elem:
                    album["album_record"] = possible_label.get_text().strip()
                else:
                    album["album_record"] = None
            else:
                album["year"] = year_text
                album["album_record"] = None
        else:
            album["year"] = None
            album["album_record"] = None
            
        # Se existir <p class="albumRecord"> de fato, usa ele como gravadora
        if album_record_elem:
            album["album_record"] = album_record_elem.get_text().strip()
            
        # Extrai lista de músicas
        album["tracks"] = self._extract_tracks(album_div)
        
        return album
        
    def _extract_tracks(self, album_div):
        """Extrai a lista de músicas de um álbum"""
        tracks = []
        track_list = album_div.find("ol", id="topMusicList")
        
        if track_list:
            track_items = track_list.find_all("li")
            for track in track_items:
                track_name_elem = track.find("a", "nameMusic")
                if track_name_elem:
                    track_name = track_name_elem.get_text().strip()
                    # Opcional: extrair URL da música para buscar letra depois
                    track_url = None
                    if track_name_elem.has_attr('href'):
                        track_url = self.base_url.rstrip('/') + track_name_elem['href']
                    
                    tracks.append({
                        "name": track_name,
                        "url": track_url
                    })
        
        return tracks

In [9]:
class LyricsExtractor:
    """Componente para extrair letras de músicas"""
    
    def __init__(self, encoding_manager=None):
        self.encoding_manager = encoding_manager
        
    def extract_lyrics(self, song_url):
        """Extrai a letra de uma música a partir da URL"""
        if not song_url:
            return None
            
        try:
            # Faz a requisição
            req = requests.get(song_url)
            
            # Usa o gerenciador de encoding se disponível
            if self.encoding_manager:
                #content = self.encoding_manager.process_content(req.content)
                content = req.content
            else:
                content = req.content
                
            # Analisa o HTML
            soup = bs4.BeautifulSoup(content, "html.parser")
            
            # No Vagalume, a letra geralmente fica em uma div com id 'lyrics'
            lyrics_div = soup.find("div", id="lyrics")
            if lyrics_div:
                lyrics = lyrics_div.get_text(separator=' ').strip()
                
                # Adiciona delay para evitar sobrecarga do servidor
                time.sleep(1.5)
                
                return lyrics
            return None
            
        except Exception as e:
            print(f"Erro ao extrair letra de {song_url}: {e}")
            return None

# Armazenamento
DataStorage: Gerencia o armazenamento dos dados extraídos

    save_to_json: Salva dados em formato JSON
    save_to_csv: Salva dados em formato CSV


In [11]:
class DataStorage:
    """Componente para armazenar dados em diferentes formatos"""
    
    def __init__(self, output_dir="output"):
        self.output_dir = output_dir
        # Cria diretório de saída se não existir
        os.makedirs(output_dir, exist_ok=True)
        
    def save_to_json(self, data, filename):
        """Salva dados em formato JSON"""
        filepath = os.path.join(self.output_dir, f"{filename}.json")
        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)
            print(f"Dados salvos em {filepath}")
            return True
        except Exception as e:
            print(f"Erro ao salvar JSON: {e}")
            return False
            
    def save_to_csv(self, data, filename):
        """Salva dados em formato CSV"""
        filepath = os.path.join(self.output_dir, f"{filename}.csv")
        try:
            # Converte para DataFrame se for uma lista de dicionários
            if isinstance(data, list) and all(isinstance(item, dict) for item in data):
                df = pd.DataFrame(data)
                df.to_csv(filepath, index=False, encoding='utf-8')
            elif isinstance(data, pd.DataFrame):
                data.to_csv(filepath, index=False, encoding='utf-8')
            else:
                raise ValueError("Formato de dados não suportado para CSV")
                
            print(f"Dados salvos em {filepath}")
            return True
        except Exception as e:
            print(f"Erro ao salvar CSV: {e}")
            return False

# Orquestrador
O Orchestrator coordena o fluxo de trabalho entre os diferentes componentes.

Principais métodos:

    process_all_artists: Processa todos os artistas da lista
    process_artist: Processa um artista específico, extraindo sua discografia e letras

In [12]:
class Orchestrator:
    """Componente orquestrador que coordena o fluxo de trabalho"""
    
    def __init__(self, artist_extractor, discography_extractor, 
                 lyrics_extractor, text_processor, data_storage):
        self.artist_extractor = artist_extractor
        self.discography_extractor = discography_extractor
        self.lyrics_extractor = lyrics_extractor
        self.text_processor = text_processor
        self.data_storage = data_storage
        
    def process_all_artists(self):
        """Processa todos os artistas disponíveis"""
        artists = self.artist_extractor.get_artists()
        
        for artist in artists:
            print(f"Processando artista: {artist}")
            self.process_artist(artist)
            
    def process_artist(self, artist):
        """Processa um artista específico"""
        # Extrai discografia
        discography, artist_name, artist_tags = self.discography_extractor.extract_discography(artist)
        
        if not discography:
            print(f"Nenhum álbum encontrado para {artist}")
            return
            
        # Salva discografia em JSON
        self.data_storage.save_to_json(discography, f"{artist}_discografia")
        
        # Prepara dados para CSV e processamento de letras
        all_songs = []
        
        for album in discography:
            for track in album["tracks"]:
                # Extrai letra se tiver URL
                lyrics = None
                if "url" in track and track["url"]:
                    lyrics = self.lyrics_extractor.extract_lyrics(track["url"])
                    
                    # Processa a letra se encontrada
                    if lyrics:
                        lyrics = self.text_processor.normalize(lyrics)
                        lyrics = self.text_processor.tokenize(lyrics)
                
                # Cria registro da música
                song_data = {
                    "artist": artist_name,
                    "tags": artist_tags,
                    "album": album["album_title"],
                    "year": album["year"],
                    "record_label": album["album_record"],
                    "title": track["name"],
                    "lyrics": lyrics if lyrics else ""
                }
                
                all_songs.append(song_data)
                break
        
        # Salva todas as músicas em CSV
        if all_songs:
            df = pd.DataFrame(all_songs)
            self.data_storage.save_to_csv(df, f"{artist}_musicas")
            
        print(f"Processamento de {artist} concluído. {len(all_songs)} músicas encontradas.")

In [ ]:
def main():
    # Inicializa componentes
    encoding_manager = EncodingManager()
    artist_extractor = ArtistExtractor('artistas.txt')
    discography_extractor = DiscographyExtractor(encoding_manager)
    lyrics_extractor = LyricsExtractor(encoding_manager)
    text_processor = TextProcessor()
    data_storage = DataStorage()
    
    # Inicializa orquestrador
    orchestrator = Orchestrator(
        artist_extractor,
        discography_extractor,
        lyrics_extractor,
        text_processor,
        data_storage
    )
    
    # Executa o processamento
    orchestrator.process_all_artists()
    
if __name__ == "__main__":
    main()